## Imports

In [3]:
import os
import numpy as np
from sklearn import svm
import pandas as pd
from sklearn.model_selection import train_test_split

from bs4 import BeautifulSoup
# The package comes built-in with Python
import re
# http://www.nltk.org/install.html
import nltk

from nltk.corpus import stopwords # Import the stop word list
stops = set(stopwords.words("english"))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics

from sklearn.grid_search import GridSearchCV
from sklearn import svm

from sklearn.ensemble import BaggingClassifier

from bs4 import BeautifulSoup 
# The package comes built-in with Python
import re
# http://www.nltk.org/install.html
import nltk 
from nltk.corpus import stopwords # Import the stop word list
stops = set(stopwords.words("english"))

/Users/andrej/.virtualenvs/summer/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/andrej/.virtualenvs/summer/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
# Import statements
import pandas as pd 
import os

In [5]:
pwd = "/Users/andrej/Documents/Mine/Zemanta/Zemanta-Data-Science-Summer-School"
os.chdir(pwd)

In [8]:
# Pre-processing step
def text_to_words(raw_text):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_text).get_text() 

    # 2. Remove non-letters
    # Find anything that is NOT a lowercase letter (a-z) or an upper case letter (A-Z), and replace it with a space
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))
    
    # There are many other things we could do to the data
    # For example, Porter Stemming and Lemmatizing (both available in NLTK) 
    # would allow us to treat "messages", "message", and "messaging" as the same word
    # which could certainly be useful.
    
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    clear_text = " ".join(meaningful_words)
    return clear_text
    #validate_text_sentiment["Message"][i] = clear_text 
    #print i
    #return( " ".join( meaningful_words ))  

## Model

In [10]:
df_train = pd.read_csv("data/task_train.csv")
df_train['cleaned_text'] = df_train.apply (lambda row: text_to_words(row.review), axis=1)

/Users/andrej/.virtualenvs/summer/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [11]:
def testModel(model, maxFeatures=3000, ngram=2):
    bigram_vectorizer = CountVectorizer(ngram_range=(1, ngram), token_pattern=r'\b\w+\b', min_df=1)
    analyze = bigram_vectorizer.build_analyzer()
    vectorizer = CountVectorizer(analyzer = analyze, tokenizer = None, preprocessor = None, stop_words = None, 
                                 max_features = maxFeatures)
    
    df_train1, df_validate1 = train_test_split(df_train, test_size=0.2)
    train_data_features = vectorizer.fit_transform(df_train1['cleaned_text'].values.astype('U'))
    train_data_features = train_data_features.toarray()
    
    model.fit(train_data_features, df_train1['sentiment'].values)
    
    test_data_features = vectorizer.transform(df_validate1['cleaned_text'].values.astype('U'))
    test_data_features = test_data_features.toarray()
    
    target_names = ['0', '1']
    print(classification_report(df_validate1['sentiment'], model.predict(test_data_features), target_names=target_names))
    

### Test Models

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

# svm.NuSVC()
# model = svm.SVC(C = 1, gamma = 0.001, kernel='rbf') # ngram=1, maxFeatures = 7000: 0.86
# RandomForestClassifier(n_estimators=100, max_features=10 )
model=LogisticRegression()
for i in range(4):
    if (i < 2):
        testModel(model, ngram=2, maxFeatures=15000)
    else:
        testModel(model, ngram=2, maxFeatures=9000)

             precision    recall  f1-score   support

          0       0.85      0.86      0.85       934
          1       0.87      0.86      0.87      1032

avg / total       0.86      0.86      0.86      1966

             precision    recall  f1-score   support

          0       0.85      0.86      0.85       985
          1       0.86      0.84      0.85       981

avg / total       0.85      0.85      0.85      1966

             precision    recall  f1-score   support

          0       0.88      0.85      0.87      1009
          1       0.85      0.87      0.86       957

avg / total       0.86      0.86      0.86      1966

             precision    recall  f1-score   support

          0       0.88      0.86      0.87      1007
          1       0.85      0.87      0.86       959

avg / total       0.87      0.87      0.87      1966



### Tune Model

In [23]:
from sklearn.ensemble import GradientBoostingClassifier
forest = RandomForestClassifier(n_estimators=100, max_features=10 )
# forest = GradientBoostingClassifier()

# param_grid = {'n_estimators': [200], 'max_features': [25,26,27, 28, 29, 30]}
param_grid = {'n_estimators': [100], 'max_features': range(10,50,10)}
grid_clf = GridSearchCV(forest, param_grid, cv=3, scoring='f1', n_jobs=-1)

In [24]:
# grid_clf.fit(train_data_features, df_train1['sentiment'].values)
grid_clf.fit(train_data_features, df_train1['sentiment'].values)
best_model = grid_clf.best_estimator_

In [2]:
# print grid_clf.best_score_
# print grid_clf.best_params_
# print grid_clf.cv_results_

#### Test F1 validity

In [26]:
test_data_features = vectorizer.transform(df_validate1['cleaned_text'].values.astype('U'))
test_data_features = test_data_features.toarray()

In [27]:
target_names = ['0', '1']
print(classification_report(df_validate1['sentiment'], best_model.predict(test_data_features), target_names=target_names))

             precision    recall  f1-score   support

          0       0.87      0.83      0.85       998
          1       0.83      0.88      0.85       968

avg / total       0.85      0.85      0.85      1966



### Test Models again

### Build Model

In [12]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [13]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
analyze = bigram_vectorizer.build_analyzer()
vectorizer = CountVectorizer(analyzer = analyze, tokenizer = None, preprocessor = None, stop_words = None, 
                             max_features = 10000)

train_data_features = vectorizer.fit_transform(df_train['cleaned_text'].values.astype('U'))
train_data_features = train_data_features.toarray()

model.fit(train_data_features, df_train['sentiment'].values)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### Export

In [17]:
df_test = pd.read_csv("data/task_test.csv")

df_test['cleaned_text'] = df_test.apply (lambda row: text_to_words(row.review), axis=1)
test_data_features = vectorizer.transform(df_test['cleaned_text'].values.astype('U'))
test_data_features = test_data_features.toarray()

df_test['predicted sentiment'] = model.predict(test_data_features)
df_test[['id', 'predicted sentiment']].to_csv("results/task_test_predicted.csv",index=False)

In [19]:
df_test[['id', 'predicted sentiment']].head()

,id,predicted sentiment
0,7166_2,0
1,6811_10,1
2,1119_1,0
3,9011_9,1
4,12106_10,1


---

---

### Bayes

In [14]:
from sklearn.linear_model import LogisticRegression

param_grid = {}
grid_clf = GridSearchCV(LogisticRegression(), param_grid, cv=10, scoring='f1', n_jobs=-1)

In [ ]:
grid_clf.fit(train_data_features, df_train1['sentiment'].values)
best_model = grid_clf.best_estimator_

In [ ]:
print grid_clf.best_score_
print grid_clf.best_params_

In [649]:
test_data_features = vectorizer.transform(df_validate1['cleaned_text'].values.astype('U'))
test_data_features = test_data_features.toarray()

In [ ]:
print(classification_report(df_validate1['sentiment'], grid_clf.predict(test_data_features), target_names=target_names))

### Forest

In [19]:
from sklearn.ensemble import GradientBoostingClassifier
forest = RandomForestClassifier(n_estimators=100, max_features=10 )
# forest = GradientBoostingClassifier()

# param_grid = {'n_estimators': [200], 'max_features': [25,26,27, 28, 29, 30]}
param_grid = {'n_estimators': [100], 'max_features': range(10,50,10)}
grid_clf = GridSearchCV(forest, param_grid, cv=3, scoring='f1', n_jobs=-1)

In [21]:
# grid_clf.fit(train_data_features, df_train1['sentiment'].values)
grid_clf.fit(train_data_features, df_train1['sentiment'].values)
best_model = grid_clf.best_estimator_

In [22]:
print grid_clf.best_score_
print grid_clf.best_params_
# print grid_clf.cv_results_

0.836435000982
{'max_features': 10, 'n_estimators': 100}


In [23]:
## Test f1 validity
test_data_features = vectorizer.transform(df_validate1['cleaned_text'].values.astype('U'))
test_data_features = test_data_features.toarray()


In [25]:
target_names = ['0', '1']
print(classification_report(df_validate1['sentiment'], best_model.predict(test_data_features), target_names=target_names))

             precision    recall  f1-score   support

          0       0.88      0.82      0.85      1013
          1       0.82      0.88      0.85       953

avg / total       0.85      0.85      0.85      1966



## Manual tests

In [327]:
testDF = df_validate.iloc[:2]
# testDF['cleaned_text'] = ['This is a super good great best movie I ever saw', 'This is a very bad movie']
test_validate_data_features = vectorizer.transform(testDF['cleaned_text'].values.astype('U'))
test_validate_data_features = test_validate_data_features.toarray()

test_y_pred = nb_model.predict(test_validate_data_features)

test_y_pred

array([0, 0])

In [12]:
df_train1, df_validate1 = train_test_split(df_train, test_size=0.2)
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
analyze = bigram_vectorizer.build_analyzer()
vectorizer = CountVectorizer(analyzer = analyze, tokenizer = None, preprocessor = None, stop_words = None, 
                             max_features = 10000)

# from sklearn.feature_extraction.text import TfidfTransformer
# transformer = TfidfTransformer(smooth_idf=False)
train_data_features = vectorizer.fit_transform(df_train1['cleaned_text'].values.astype('U'))
train_data_features = train_data_features.toarray()
y_actual = df_train.sentiment.values
dist = np.sum(train_data_features, axis=0)

dist.sort()
print dist

[   15    15    15 ...  8295 12729 13673]
